<a href="https://colab.research.google.com/github/josephgiovanelli/PPS-17-ese/blob/master/it/unibo/abds/smart_irrigation/notebooks/data_acquisition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Defining tensorflow version
%tensorflow_version 2.x
import tensorflow
print(tensorflow.__version__)

2.3.0


In [12]:
# Importing main libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
# Importing the sql dump
from google.colab import files
uploaded = files.upload()

Saving september.csv to september (3).csv


In [14]:
# Reading the sql dump
data = pd.read_csv('september.csv')
data

,plantNum,yy,xx,unit,date,rounded_timestamp,aggregated_value
0,1,-60.0,0.0,cbar,2020-09-01 00:00:00,1598911200,-78.711091
1,1,-60.0,30.0,cbar,2020-09-01 00:00:00,1598911200,-85.089666
2,1,-60.0,60.0,cbar,2020-09-01 00:00:00,1598911200,-58.067100
3,1,-60.0,90.0,cbar,2020-09-01 00:00:00,1598911200,-34.166775
4,1,-40.0,0.0,cbar,2020-09-01 00:00:00,1598911200,-80.806075
...,...,...,...,...,...,...,...
22185,2,-20.0,60.0,cbar,2020-10-01 00:00:00,1601586000,-267.875806
22186,2,-20.0,90.0,cbar,2020-10-01 00:00:00,1601586000,-21.616472
22187,2,NaN,NaN,°C,2020-10-01 00:00:00,1601586000,17.150000
22188,2,NaN,NaN,l,2020-10-01 00:00:00,1601586000,0.000000


In [15]:
# Defining constants
X_START, X_STOP, X_STEP = 0, 90, 30
Y_START, Y_STOP, Y_STEP = -60, -20, 20
UNITS = {"dripper": "l", "pluv_curr": "mm", "air_temp": "°C", "grnd_water": "cbar"}

# Each record reprensents a feature of the specified (plantNum, timestamp)
# For each record, it has been created an ad-hoc column, with the name of the feature in question, and the value contained in aggregated_value
for key in UNITS.keys():
  if key == "grnd_water":
    for y in range(Y_START, Y_STOP + 1, Y_STEP):
      for x in range(X_START, X_STOP + 1, X_STEP):
        data[key + "_" + str(-y) + "_" + str(x)] = data.apply(lambda df: df.aggregated_value if (UNITS[key] in df.unit and df.yy == y and df.xx == x) else np.nan , axis=1)
  else:
    data[key] = data.apply(lambda df: df.aggregated_value if UNITS[key] in df.unit else np.nan , axis=1)

In [16]:
# Merging all the values related to a certain (plantNum, timestamp) tuple in one record
data = data.groupby(by=[data.plantNum, data.rounded_timestamp]).max()

# Dropping useless columns
data = data.drop(['date', 'yy', 'xx', 'unit', 'aggregated_value'], axis=1)
data

dripper  ...  grnd_water_20_90
plantNum rounded_timestamp           ...                  
1        1598911200             0.0  ...       -637.664231
         1598914800             0.0  ...       -639.883380
         1598918400             0.0  ...       -643.210776
         1598922000             0.0  ...       -645.783084
         1598925600             0.0  ...       -647.999801
...                             ...  ...               ...
2        1601571600             0.0  ...        -21.041212
         1601575200             0.0  ...        -21.041212
         1601578800             0.0  ...        -21.041212
         1601582400             0.0  ...        -21.616472
         1601586000             0.0  ...        -21.616472

[1488 rows x 15 columns]

In [17]:
HOUR_TO_SECONDS = 3600
import statistics

# Imputing missing values: each missing value is imputed by applying the mean between the values of the same feature but belonging to the hour before and the hour after
for col in data.columns:
  for index, row in data[data[col].isnull()].iterrows():
    #print(data.loc[index, col])
    data.loc[index, col] = statistics.mean([data.loc[(index[0], index[1] - HOUR_TO_SECONDS), col], data.loc[(index[0], index[1] - HOUR_TO_SECONDS), col]])
    #print(data.loc[index, col])
    #print()


In [18]:
# Checking if there is any NaN value left
data.isnull().values.any()

False

In [38]:
# Selecting data of just one plant
plant_1 = data.loc[[1]]
plant_2 = data.loc[[2]]

# Converting pd dataframe to a matrix
sets = {"train": np.asmatrix(plant_1), "test": np.asmatrix(plant_2)}

In [41]:
# Creating a data structure with 90 timestamps and 1 output
X = {"train": [], "test": []}
y = {"train": [], "test": []}

# Number of hours we want top predict into the future
n_future = 4  
# Number of past hours we want to use to predict the future
n_past = 24     

# Creating the train and test sets 
# e.g. with timestamps 0-23 we are going to predict the timestamps 24-27, 
# then with timestamps 1-24 we are going to predict the timestamps 25-28,  
# until with timestamps 690-713 we are going to predict the timestamps 714-717.  
for i in ["train", "test"]:
  for j in range(n_past, len(sets[i]) - n_future + 1):
      X[i].append(sets[i][j - n_past : j - 1, :])
      y[i].append(sets[i][j : j + n_future - 1, 3:])
  X[i], y[i] = np.array(X[i]), np.array(y[i])
  print('X {} shape = {}.'.format(i, X[i].shape))
  print('y {} shape = {}.'.format(i, y[i].shape))
  print()

X train shape = (717, 23, 15).
y train shape = (717, 3, 12).

X test shape = (717, 23, 15).
y test shape = (717, 3, 12).

